### __Machine Learning Models: Regression Test v2__

In [1]:
# import libraries
import numpy as np
import pandas as pd
import random
import io
import requests
from datetime import datetime, timezone, timedelta
import datetime as dt
import time
from collections import Counter
from PIL import Image
import PIL.Image

# import matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.image as mpimg

# import seaborn
import seaborn as sns

# import plotly graph objects
import plotly.graph_objs as go
import plotly.express as px

# import scipy.cluster.hierarchy
import scipy
import scipy.linalg as la
import scipy.cluster.hierarchy as sch
from scipy.cluster.vq import whiten, kmeans, vq
from scipy.stats import multivariate_normal
from scipy.stats import loguniform
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

# import category encoders
#import category_encoders as ce

import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
import sklearn.model_selection as model_selection
import sklearn.model_selection as cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import average_precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_precision_recall_curve, plot_confusion_matrix
from sklearn.feature_selection import RFE
from sklearn import linear_model, datasets
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.svm import SVR
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import sklearn.gaussian_process as gp
from sklearn.neural_network import MLPClassifier

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

#for showing 2D plot
%matplotlib inline 

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Training Dataset import
train_df = pd.read_csv('training_for_regmodel_65k.csv')
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65083 entries, 0 to 65082
Columns: 853 entries, fl_date to target
dtypes: float64(8), int64(844), object(1)
memory usage: 423.6+ MB


,fl_date,crs_elapsed_time,target_class,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,...,crs_arr_hours_22,crs_arr_hours_23,crs_arr_hours_3,crs_arr_hours_4,crs_arr_hours_5,crs_arr_hours_6,crs_arr_hours_7,crs_arr_hours_8,crs_arr_hours_9,target
0,2019-05-01,66.0,1,0.321053,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,...,0,0,0,0,0,1,0,0,0,4.0
1,2019-05-01,293.0,1,0.218119,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,...,0,0,0,0,0,0,0,0,0,10.0
2,2019-05-01,72.0,1,0.321053,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,...,0,0,0,0,0,0,1,0,0,2.0
3,2019-05-01,123.0,1,0.290223,5.0,22455,8618120.0,4.309060e+07,5318166.6,2.659083e+07,...,0,0,0,0,0,0,0,0,0,11.0
4,2019-05-01,182.0,1,0.188597,5.0,22455,8618120.0,4.309060e+07,5318166.6,2.659083e+07,...,0,0,0,0,0,0,0,0,0,85.0


In [3]:
# start time to measure the time of the program execution
start_time = time.time()

# remove variables related to dummy_features from train_df
# also remove 'flights' variable
remove_lower_features = ['fl_date']
train_df = train_df.drop(remove_lower_features, axis=1)

# check
train_df.info()
train_df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65083 entries, 0 to 65082
Columns: 852 entries, crs_elapsed_time to target
dtypes: float64(8), int64(844)
memory usage: 423.1 MB


,crs_elapsed_time,target_class,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,year_2018,...,crs_arr_hours_22,crs_arr_hours_23,crs_arr_hours_3,crs_arr_hours_4,crs_arr_hours_5,crs_arr_hours_6,crs_arr_hours_7,crs_arr_hours_8,crs_arr_hours_9,target
0,66.0,1,0.321053,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,0,...,0,0,0,0,0,1,0,0,0,4.0
1,293.0,1,0.218119,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,0,...,0,0,0,0,0,0,0,0,0,10.0
2,72.0,1,0.321053,5.0,79895,598183733.4,2.990919e+09,303232966.2,1.516165e+09,0,...,0,0,0,0,0,0,1,0,0,2.0
3,123.0,1,0.290223,5.0,22455,8618120.0,4.309060e+07,5318166.6,2.659083e+07,0,...,0,0,0,0,0,0,0,0,0,11.0
4,182.0,1,0.188597,5.0,22455,8618120.0,4.309060e+07,5318166.6,2.659083e+07,0,...,0,0,0,0,0,0,0,0,0,85.0




--- 0.2566404342651367 seconds ---


#### __Standard Scale Numerical Variables__
> - crs_elapsed_time
> - speed_passengers
> - mean_passengers
> - sum_passengers
> - total_cost:mean_fuel
> - total_cost:sum_fuel
> - total_gallons:mean_fuel
> - total_gallons:sum_fuel

In [4]:
train_columns = train_df.columns.tolist()
train_columns

['crs_elapsed_time',
 'target_class',
 'speed_passengers',
 'mean_passengers',
 'sum_passengers',
 'total_cost:mean_fuel',
 'total_cost:sum_fuel',
 'total_gallons:mean_fuel',
 'total_gallons:sum_fuel',
 'year_2018',
 'year_2019',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'day_of_week_0',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'mkt_unique_carrier_AA',
 'mkt_unique_carrier_AS',
 'mkt_unique_carrier_B6',
 'mkt_unique_carrier_DL',
 'mkt_unique_carrier_F9',
 'mkt_unique_carrier_G4',
 'mkt_unique_carrier_HA',
 'mkt_unique_carrier_NK',
 'mkt_unique_carrier_UA',
 'mkt_unique_carrier_VX',
 'mkt_unique_carrier_WN',
 'origin_airport_id_10135',
 'origin_airport_id_10136',
 'origin_airport_id_10140',
 'origin_airport_id_10141',
 'origin_airport_id_10146',
 'origin_airport_id_10154',
 'origin_airport_id_10155',
 'origin_airpor

In [5]:
# start time to measure the time of the program execution
start_time = time.time()

# get the numerical features & review their stats
feature_names_num = ['crs_elapsed_time', 'speed_passengers', 
                    'mean_passengers', 'sum_passengers',
                    'total_cost:mean_fuel', 'total_cost:sum_fuel',
                    'total_gallons:mean_fuel', 'total_gallons:sum_fuel']

var_scale = train_df[feature_names_num]
var_scale.describe()

# standard scale the numerical variables & make them into pd.dataframe
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(var_scale), columns = feature_names_num)
df_scale

# replace the numerical variables with the scaled variables
train_df2 = train_df.drop(feature_names_num, axis=1)

# combine dummy variables with work_df
train_df = pd.concat([train_df2, df_scale], axis=1)

# rearrange the columns in work_df3
train_df = train_df[train_columns]
train_df.info()
train_df.head()

# check
train_df.info()
train_df.head()
len(train_df.columns)
train_df.columns.tolist()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,crs_elapsed_time,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel
count,65083.000000,65083.000000,65083.000000,65083.000000,6.508300e+04,6.508300e+04,6.508300e+04,6.508300e+04
mean,138.735476,0.274128,6.571900,64175.742452,2.360798e+08,1.123252e+09,1.280723e+08,6.091810e+08
std,71.847516,0.771201,3.400552,55123.075956,2.317318e+08,1.117408e+09,1.261025e+08,6.074206e+08
min,-112.000000,0.000000,1.000000,422.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,87.000000,0.173545,4.000000,18771.000000,7.937670e+06,3.907853e+07,5.262524e+06,2.594027e+07
50%,120.000000,0.224577,7.000000,45624.000000,1.327268e+08,6.636341e+08,6.828546e+07,3.399067e+08
75%,169.000000,0.307466,10.000000,102200.000000,4.721148e+08,2.097587e+09,2.688937e+08,1.182938e+09
max,665.000000,72.181818,12.000000,235632.000000,7.342249e+08,3.671125e+09,3.385212e+08,1.692606e+09


,crs_elapsed_time,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel
0,-1.012367,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184
1,2.147127,-0.072626,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184
2,-0.928856,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184
3,-0.219014,0.020871,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129
4,0.602176,-0.110906,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129
...,...,...,...,...,...,...,...,...
65078,-0.817508,-0.136653,1.302181,1.905120,0.356276,0.135409,0.300544,0.090061
65079,0.017600,-0.097453,1.302181,1.905120,0.356276,0.135409,0.300544,0.090061
65080,0.156785,0.064486,1.302181,1.905120,0.356276,0.135409,0.300544,0.090061
65081,-0.051992,-0.083426,1.302181,1.905120,0.356276,0.135409,0.300544,0.090061


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65083 entries, 0 to 65082
Columns: 852 entries, crs_elapsed_time to target
dtypes: float64(9), int64(843)
memory usage: 423.1 MB


,crs_elapsed_time,target_class,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,year_2018,...,crs_arr_hours_22,crs_arr_hours_23,crs_arr_hours_3,crs_arr_hours_4,crs_arr_hours_5,crs_arr_hours_6,crs_arr_hours_7,crs_arr_hours_8,crs_arr_hours_9,target
0,-1.012367,1,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,1,0,0,0,4.0
1,2.147127,1,-0.072626,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,0,0,0,0,10.0
2,-0.928856,1,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,0,1,0,0,2.0
3,-0.219014,1,0.020871,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129,0,...,0,0,0,0,0,0,0,0,0,11.0
4,0.602176,1,-0.110906,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129,0,...,0,0,0,0,0,0,0,0,0,85.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65083 entries, 0 to 65082
Columns: 852 entries, crs_elapsed_time to target
dtypes: float64(9), int64(843)
memory usage: 423.1 MB


,crs_elapsed_time,target_class,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,year_2018,...,crs_arr_hours_22,crs_arr_hours_23,crs_arr_hours_3,crs_arr_hours_4,crs_arr_hours_5,crs_arr_hours_6,crs_arr_hours_7,crs_arr_hours_8,crs_arr_hours_9,target
0,-1.012367,1,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,1,0,0,0,4.0
1,2.147127,1,-0.072626,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,0,0,0,0,10.0
2,-0.928856,1,0.060848,-0.462252,0.285169,1.562611,1.671441,1.389045,1.493184,0,...,0,0,0,0,0,0,1,0,0,2.0
3,-0.219014,1,0.020871,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129,0,...,0,0,0,0,0,0,0,0,0,11.0
4,0.602176,1,-0.110906,-0.462252,-0.756871,-0.981580,-0.966675,-0.973455,-0.959129,0,...,0,0,0,0,0,0,0,0,0,85.0


852

['crs_elapsed_time',
 'target_class',
 'speed_passengers',
 'mean_passengers',
 'sum_passengers',
 'total_cost:mean_fuel',
 'total_cost:sum_fuel',
 'total_gallons:mean_fuel',
 'total_gallons:sum_fuel',
 'year_2018',
 'year_2019',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'day_of_week_0',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'mkt_unique_carrier_AA',
 'mkt_unique_carrier_AS',
 'mkt_unique_carrier_B6',
 'mkt_unique_carrier_DL',
 'mkt_unique_carrier_F9',
 'mkt_unique_carrier_G4',
 'mkt_unique_carrier_HA',
 'mkt_unique_carrier_NK',
 'mkt_unique_carrier_UA',
 'mkt_unique_carrier_VX',
 'mkt_unique_carrier_WN',
 'origin_airport_id_10135',
 'origin_airport_id_10136',
 'origin_airport_id_10140',
 'origin_airport_id_10141',
 'origin_airport_id_10146',
 'origin_airport_id_10154',
 'origin_airport_id_10155',
 'origin_airpor



--- 3.580524206161499 seconds ---


In [6]:
# remove 'target_class'
train_df = train_df.drop(columns=['target_class'])

In [8]:
train_df.iloc[:, 850]

0         4.0
1        10.0
2         2.0
3        11.0
4        85.0
         ... 
65078   -12.0
65079   -23.0
65080   -14.0
65081    -7.0
65082   -19.0
Name: target, Length: 65083, dtype: float64

#### __Regression Models - Initialize__
> - Linear Regression
> - Ridge Regression
> - Lasso Regression
> - Decision Tree Regressor: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
> - Random Forest Regressor: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
> - KNN: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
> - XGBoost: https://machinelearningmastery.com/xgboost-for-regression/

> - Hyperparameter Tuning: Randomizer https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/

In [9]:
# Assign X & y
X = train_df.iloc[:, :850].values
y = train_df.iloc[:, 850].values

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [10]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

LinearRegression()



--- 17.220505475997925 seconds ---


In [11]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
rr = Ridge()
rr.fit(X_train, y_train)
y_pred_rr = rr.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

Ridge()



--- 1.7980639934539795 seconds ---


In [12]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
ls = Lasso()
ls.fit(X_train, y_train)
y_pred_ls = ls.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

Lasso()



--- 2.8751578330993652 seconds ---


In [13]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
dr = DecisionTreeRegressor(random_state=0)
dr.fit(X_train, y_train)
y_pred_dr = dr.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

DecisionTreeRegressor(random_state=0)



--- 32.81974649429321 seconds ---


In [14]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
rfr = RandomForestRegressor(random_state=0)
rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

RandomForestRegressor(random_state=0)



--- 2183.094354867935 seconds ---


In [15]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

KNeighborsRegressor()



--- 158.57074522972107 seconds ---


In [16]:
# start time to measure the time of the program execution
start_time = time.time()

# Fit Random Forest Model
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)



--- 151.21321749687195 seconds ---


In [17]:
# start time to measure the time of the program execution
start_time = time.time()

# function for measuring the performance of regression models
def regression_metrics(y_test, X_test, y_pred_lst, n, k):
    
    linear_r = []
    ridge = []
    lasso = []
    decision_r = []
    rf_r = []
    knn_r = []
    xgb_r = []
    
    model_lst = [linear_r, ridge, lasso, decision_r, rf_r, knn_r, 
                 xgb_r]
    
    n= 0
    
    # for each y_pred in the list, compute the metrics
    for item in y_pred_lst:
        r2_test = r2_score(y_test, item)
        adjusted_R2 = 1 - ((n - 1) / (n - k - 1)) * (1 - r2_test)
        mae_test = mean_absolute_error(y_test, item)
        mse_test = mean_squared_error(y_test, item)
        rmse_test = np.sqrt(mean_squared_error(y_test, item))
        
        model_lst[n] = [r2_test, adjusted_R2, mae_test, mse_test, rmse_test]
        n += 1
        
    df_metrics_summary = pd.DataFrame({'linear_reg': model_lst[0], 'ridge': model_lst[1], 'lasso': model_lst[2],
                                    'decision_reg': model_lst[3], 'random_forest': model_lst[4], 
                                    'knn': model_lst[5], 'xgboost': model_lst[6]},
                                      index = ['R2', 'Adjusted_R2','MAE', 'MSE', 'RMSE'])

    return df_metrics_summary

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")



--- 0.0021820068359375 seconds ---


In [18]:
# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

n = len(X_test)
k = len(train_df.columns.tolist())-1

y_pred_lst = [y_pred_lr, y_pred_rr, y_pred_ls, y_pred_dr, y_pred_rfr,
              y_pred_knn, y_pred_xgb]

regression_metrics(y_test, X_test, y_pred_lst, n, k)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")



--- 5.472023010253906 seconds ---


,linear_reg,ridge,lasso,decision_reg,random_forest,knn,xgboost
R2,-2.564856e+15,0.004680,0.000914,-0.804665,-0.091168,-0.122313,-0.049703
Adjusted_R2,-3.013932e+12,1.000000,1.001177,1.004256,1.003865,1.005306,1.006211
MAE,4.925865e+07,29.535826,29.852060,39.630522,30.275294,32.793992,29.802311
MSE,9.129622e+18,3542.847229,3556.251202,6423.716313,3884.020465,3994.878694,3736.422718
RMSE,3.021526e+09,59.521821,59.634312,80.148090,62.321910,63.205053,61.126285




--- 5.706674337387085 seconds ---


#### __RFE: Feature Selection__

In [19]:
X.shape[1]
len(train_df.columns.tolist())-1

850

850

In [ ]:
# start time to measure the time of the program execution
start_time = time.time()

# Feature Selections through RFE
for n in range(X.shape[1]):
    # define RFE
    rfe = RFE(estimator=XGBRegressor(), n_features_to_select=n)
    # fit RFE
    rfe.fit(X_train, y_train)
    
    y_pred_xgb0 = rfe.predict(X_test)
    r2_test0 = r2_score(y_test, y_pred_xgb0)
    
    if r2_test0 > 0:
        # summarize all features
        print('Column: %d, Selected %s, Rank: %.3f' % (n, rfe.support_[n], rfe.ranking_[n]))

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")